In [2]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 352.00 KiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [3]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=3,
    lora_alpha=9,
    lora_dropout=0.1,
    bias='lora_only',
    target_modules=['query', 'key', 'value'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 787,204 || all params: 125,408,264 || trainable%: 0.6277


In [4]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [5]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.366100,1.316062,0.681250
200,0.873800,0.386398,0.865625
300,0.371000,0.340972,0.887500
400,0.308400,0.332698,0.896875
500,0.318600,0.323757,0.901563
600,0.299500,0.338370,0.892188
700,0.308700,0.320299,0.904687
800,0.318100,0.310010,0.906250
900,0.290500,0.313633,0.904687
1000,0.290400,0.308268,0.904687


In [9]:
training_args = TrainingArguments(
    # Core training configs
    max_steps=3000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args, checkpoint='/kaggle/working/saved_models/checkpoint-2000')

Starting training...
Resuming from checkpoint: /kaggle/working/saved_models/checkpoint-2000


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss,Validation Loss,Accuracy
2100,0.301800,0.301213,0.907813
2200,0.294500,0.299012,0.904687
2300,0.269800,0.303094,0.901563
2400,0.301100,0.300189,0.904687
2500,0.276700,0.298121,0.909375
2600,0.265200,0.298179,0.909375
2700,0.263100,0.300208,0.909375
2800,0.266600,0.297671,0.907813
2900,0.277500,0.298599,0.909375
3000,0.265900,0.298503,0.907813


In [10]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2500'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:59<00:00,  4.24it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2500.csv


In [11]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-3000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3661,1.316062,0.361250,0.681250,0.000048,0.013405,0.050038,1.038021,-0.320000,0.530275
1,200,0.8738,0.386398,0.753571,0.865625,0.000045,0.026810,0.487402,2.261400,-0.112054,0.870552
2,300,0.3710,0.340972,0.871875,0.887500,0.000043,0.040214,0.030028,1.088066,-0.015625,0.982394
3,400,0.3084,0.332698,0.894643,0.896875,0.000040,0.053619,-0.024298,0.926967,-0.002232,0.997511
4,500,0.3186,0.323757,0.891518,0.901563,0.000037,0.067024,-0.005157,0.984072,-0.010045,0.988859
5,600,0.2995,0.338370,0.899554,0.892188,0.000035,0.080429,-0.038870,0.885125,0.007366,1.008256
6,700,0.3087,0.320299,0.897768,0.904687,0.000033,0.093834,-0.011599,0.963787,-0.006920,0.992351
7,800,0.3181,0.310010,0.894196,0.906250,0.000030,0.107239,0.008090,1.026095,-0.012054,0.986700
8,900,0.2905,0.313633,0.904464,0.904687,0.000028,0.120643,-0.023133,0.926243,-0.000223,0.999753
9,1000,0.2904,0.308268,0.901786,0.904687,0.000025,0.134048,-0.017868,0.942038,-0.002902,0.996792
